# IMDB 5000 Movie Dataset Data Cleaning

=======================================================================================================

**AUTHOR**: Mengshan Jin

**CREATION DATE**: 07/31/2017

=======================================================================================================

**PROGRAM DESCRIPTION**: Data cleaning on IMDB 5000 Movie Dataset

**INPUT DATASETS**: 01_Data/Inputs/movie_metadata.csv

**OUTPUT DATASETS**: 01_Data/Outputs/imdb_modern_cleaned.csv


=======================================================================================================

**PROGRAM CHANGE HISTORY**

Date|Author|Change|
----|------|------|

# Section 0: Import packages

In [1]:
# Data structure
import numpy as np
import pandas as pd
from itertools import chain
import copy
from collections import Counter

# Plotting
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns; sns.set(style="ticks", color_codes=True)

# Custom support functions
import sys    
sys.path.insert(0,"../03_Feature_Engineering/sklearn-support/")
import support as sup

# sklearn
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, Imputer, FunctionTransformer
from sklearn.pipeline import Pipeline

# Section 1: Read data

In [3]:
imdb = pd.read_csv("../01_Data/Inputs/movie_metadata.csv")

In [4]:
imdb['color'] = imdb['color'].str.strip()

# Section 2: Impute missing values

## Part 1: Inspection

In [5]:
imdb.shape

(5043, 28)

In [4]:
imdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5043 entries, 0 to 5042
Data columns (total 28 columns):
color                        5024 non-null object
director_name                4939 non-null object
num_critic_for_reviews       4993 non-null float64
duration                     5028 non-null float64
director_facebook_likes      4939 non-null float64
actor_3_facebook_likes       5020 non-null float64
actor_2_name                 5030 non-null object
actor_1_facebook_likes       5036 non-null float64
gross                        4159 non-null float64
genres                       5043 non-null object
actor_1_name                 5036 non-null object
movie_title                  5043 non-null object
num_voted_users              5043 non-null int64
cast_total_facebook_likes    5043 non-null int64
actor_3_name                 5020 non-null object
facenumber_in_poster         5030 non-null float64
plot_keywords                4890 non-null object
movie_imdb_link              5043 non-

In [6]:
imdb_modern = imdb.loc[(imdb['title_year'] > 1980) & (imdb['language'] == "English")]

In [7]:
imdb_modern.shape

(4339, 28)

In [8]:
imdb_modern.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4650 entries, 0 to 5042
Data columns (total 28 columns):
color                        4635 non-null object
director_name                4650 non-null object
num_critic_for_reviews       4609 non-null float64
duration                     4638 non-null float64
director_facebook_likes      4650 non-null float64
actor_3_facebook_likes       4634 non-null float64
actor_2_name                 4640 non-null object
actor_1_facebook_likes       4643 non-null float64
gross                        4040 non-null float64
genres                       4650 non-null object
actor_1_name                 4643 non-null object
movie_title                  4650 non-null object
num_voted_users              4650 non-null int64
cast_total_facebook_likes    4650 non-null int64
actor_3_name                 4634 non-null object
facenumber_in_poster         4637 non-null float64
plot_keywords                4510 non-null object
movie_imdb_link              4650 non-

In [8]:
imdb_modern.to_csv("../01_Data/Inputs/movie_metadata_modern.csv", index=False)

In [9]:
imdb_modern = pd.read_csv("../01_Data/Inputs/movie_metadata_modern.csv")

### Question: Depending on title_year?

In [10]:
imdb_modern.loc[imdb_modern['aspect_ratio'].isnull()]['title_year'].value_counts()
# It looks pretty random, can impute with median

2014.0    51
2015.0    43
2013.0    22
2016.0    19
2012.0    18
2011.0    15
2010.0    13
2009.0    12
2004.0     9
2005.0     7
2006.0     6
2007.0     6
1997.0     6
1999.0     5
2000.0     4
2003.0     4
1998.0     4
2008.0     3
2002.0     3
1982.0     2
1990.0     2
1983.0     2
1994.0     1
1995.0     1
1985.0     1
1987.0     1
1996.0     1
Name: title_year, dtype: int64

In [11]:
imdb_modern.loc[imdb_modern['color'].isnull()]['title_year'].value_counts()
# same here

2012.0    2
2011.0    2
2015.0    2
2014.0    2
1990.0    1
2013.0    1
2010.0    1
Name: title_year, dtype: int64

## Part 2: Preprocessing
1. aspect_ratio: replace 16 with 1.85
2. content_rating: replace "Not Rated" and NaN with "Unrated"
3. remove movie_imdb_link

In [12]:
imdb_modern['aspect_ratio'] = imdb_modern['aspect_ratio'].apply(lambda x: 1.85 if x == 16 else x)

In [13]:
imdb_modern['aspect_ratio'].describe()

count    4078.000000
mean        2.101623
std         0.268843
min         1.180000
25%         1.850000
50%         2.350000
75%         2.350000
max         2.760000
Name: aspect_ratio, dtype: float64

In [14]:
imdb_modern['content_rating'] = imdb_modern['content_rating'].apply(lambda x: "Unrated" if x == "Not Rated" or pd.isnull(x) else x)

In [15]:
imdb_modern['content_rating'].value_counts()

R          1941
PG-13      1415
PG          607
Unrated     271
G            83
X             8
NC-17         6
TV-G          3
TV-PG         3
TV-14         2
Name: content_rating, dtype: int64

In [16]:
imdb_modern = imdb_modern.drop('movie_imdb_link', 1)

## Part 3: Handle genres and plot_keywords

In [17]:
dtmp = imdb_modern['genres'].apply(lambda x: '|'.join(pd.Series(x))).str.get_dummies()

In [18]:
imdb_modern = pd.concat([imdb_modern, dtmp], axis=1)
imdb_modern = imdb_modern.drop('genres', 1)

### plot_keywords
Only keep dummy columns for plot_keywords that are associated with more than 30 movies.

In [19]:
plot_keywords = Counter([item for sublist in list(imdb_modern['plot_keywords'].str.split('|')) if sublist is not np.nan for item in sublist])
plot_keywords = pd.DataFrame(plot_keywords.most_common()).set_index([0], drop=True)
del plot_keywords.index.name
plot_keywords.columns = ['Count']
plot_keywords = plot_keywords.loc[plot_keywords['Count']>30].index.tolist()

In [20]:
ktmp = imdb_modern['plot_keywords'].apply(lambda x: '|'.join(pd.Series(x)) if pd.notnull(x) else None).str.get_dummies()
ktmp = ktmp[plot_keywords]
ktmp.columns = ['keyword_' + col for col in ktmp.columns]

In [21]:
imdb_modern = pd.concat([imdb_modern, ktmp], axis=1)
imdb_modern = imdb_modern.drop('plot_keywords', 1)

In [22]:
imdb_modern.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,actor_1_name,...,keyword_assassin,keyword_girl,keyword_violence,keyword_critically bashed,keyword_actor,keyword_blood,keyword_neighbor,keyword_teacher,keyword_doctor,keyword_new york
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,CCH Pounder,...,0,0,0,0,0,0,0,0,0,0
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Johnny Depp,...,0,0,0,0,0,0,0,0,0,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Christoph Waltz,...,0,0,0,0,0,0,0,0,0,0
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Tom Hardy,...,0,0,0,0,0,0,0,0,0,0
4,Color,Andrew Stanton,462.0,132.0,475.0,530.0,Samantha Morton,640.0,73058679.0,Daryl Sabara,...,0,0,0,0,0,0,0,0,0,0


In [23]:
imdb_modern.set_index('movie_title', inplace=True)

In [26]:
imdb_modern = imdb_modern.drop("language", 1)

In [27]:
imdb_modern.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,actor_1_name,...,keyword_assassin,keyword_girl,keyword_violence,keyword_critically bashed,keyword_actor,keyword_blood,keyword_neighbor,keyword_teacher,keyword_doctor,keyword_new york
movie_title,,,,,,,,,,,,,,,,,,,,,
Avatar,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,CCH Pounder,...,0,0,0,0,0,0,0,0,0,0
Pirates of the Caribbean: At World's End,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Johnny Depp,...,0,0,0,0,0,0,0,0,0,0
Spectre,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Christoph Waltz,...,0,0,0,0,0,0,0,0,0,0
The Dark Knight Rises,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Tom Hardy,...,0,0,0,0,0,0,0,0,0,0
John Carter,Color,Andrew Stanton,462.0,132.0,475.0,530.0,Samantha Morton,640.0,73058679.0,Daryl Sabara,...,0,0,0,0,0,0,0,0,0,0


## Part 4: Use Transformers to impute missing values

In [28]:
clean_pipe = Pipeline([
    ('impute_with_unknown', sup.UnknownImputer(['actor_1_name', 'actor_2_name', 'actor_3_name', 'country', 'director_name'])),
    ('combine_small_categories', sup.SmallCategoryCombiner(['country', 'actor_1_name', 'actor_2_name',
                                                            'actor_3_name', 'director_name'])),
    ('convert_to_category', sup.DtypeConverter(['color', 'content_rating', 'country', 'actor_1_name', 
                                                'actor_2_name', 'actor_3_name', 'director_name'])),
    ('create_dummies', sup.CategoricalTransformer()),
    ('impute_missings', Imputer(strategy='median'))
])

In [29]:
imdb_modern_cleaned = copy.deepcopy(imdb_modern)
for i in range(len(clean_pipe.steps)-1):
    imdb_modern_cleaned = clean_pipe.steps[i][1].fit_transform(imdb_modern_cleaned)

In [30]:
imdb_modern_cleaned.shape

(4339, 255)

In [31]:
imdb_modern_cleaned.head()

,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_1_facebook_likes,gross,num_voted_users,cast_total_facebook_likes,facenumber_in_poster,num_user_for_reviews,...,content_rating_G,content_rating_NC-17,content_rating_PG,content_rating_PG-13,content_rating_R,content_rating_TV-14,content_rating_TV-G,content_rating_TV-PG,content_rating_Unrated,content_rating_X
movie_title,,,,,,,,,,,,,,,,,,,,,
Avatar,723.0,178.0,0.0,855.0,1000.0,760505847.0,886204,4834,0.0,3054.0,...,0,0,0,1,0,0,0,0,0,0
Pirates of the Caribbean: At World's End,302.0,169.0,563.0,1000.0,40000.0,309404152.0,471220,48350,0.0,1238.0,...,0,0,0,1,0,0,0,0,0,0
Spectre,602.0,148.0,0.0,161.0,11000.0,200074175.0,275868,11700,1.0,994.0,...,0,0,0,1,0,0,0,0,0,0
The Dark Knight Rises,813.0,164.0,22000.0,23000.0,27000.0,448130642.0,1144337,106759,0.0,2701.0,...,0,0,0,1,0,0,0,0,0,0
John Carter,462.0,132.0,475.0,530.0,640.0,73058679.0,212204,1873,1.0,738.0,...,0,0,0,1,0,0,0,0,0,0


In [32]:
imdb_modern_cleaned_final = pd.DataFrame(clean_pipe.steps[4][1].fit_transform(imdb_modern_cleaned), columns=imdb_modern_cleaned.columns, index=imdb_modern_cleaned.index)

In [33]:
imdb_modern_cleaned_final.head()

,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_1_facebook_likes,gross,num_voted_users,cast_total_facebook_likes,facenumber_in_poster,num_user_for_reviews,...,content_rating_G,content_rating_NC-17,content_rating_PG,content_rating_PG-13,content_rating_R,content_rating_TV-14,content_rating_TV-G,content_rating_TV-PG,content_rating_Unrated,content_rating_X
movie_title,,,,,,,,,,,,,,,,,,,,,
Avatar,723.0,178.0,0.0,855.0,1000.0,760505847.0,886204.0,4834.0,0.0,3054.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
Pirates of the Caribbean: At World's End,302.0,169.0,563.0,1000.0,40000.0,309404152.0,471220.0,48350.0,0.0,1238.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
Spectre,602.0,148.0,0.0,161.0,11000.0,200074175.0,275868.0,11700.0,1.0,994.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
The Dark Knight Rises,813.0,164.0,22000.0,23000.0,27000.0,448130642.0,1144337.0,106759.0,0.0,2701.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
John Carter,462.0,132.0,475.0,530.0,640.0,73058679.0,212204.0,1873.0,1.0,738.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
imdb_modern_cleaned_final.duplicated().sum()

38

In [38]:
imdb.duplicated().sum()

45

In [39]:
imdb_modern_cleaned_final = imdb_modern_cleaned_final.drop_duplicates()

In [40]:
imdb_modern_cleaned_final.to_csv("../01_Data/Outputs/imdb_modern_cleaned.csv", index=True)